<a href="https://colab.research.google.com/github/NabzT7/ASS4/blob/main/Paqueta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
unlabelled_data=pd.read_csv('unlabelled_test_data.csv')
training_data=pd.read_csv('training_data.csv')
training_data.head(2)



,id,sentence,difficulty
0,0,Les coûts kilométriques réels peuvent diverger...,C1
1,1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1


In [27]:
unlabelled_data.head(2)

,id,sentence
0,0,Nous dûmes nous excuser des propos que nous eû...
1,1,Vous ne pouvez pas savoir le plaisir que j'ai ...


In [28]:
print(training_data.info())
print(training_data.describe())

print(unlabelled_data.info())
print(unlabelled_data.describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4800 entries, 0 to 4799
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          4800 non-null   int64 
 1   sentence    4800 non-null   object
 2   difficulty  4800 non-null   object
dtypes: int64(1), object(2)
memory usage: 112.6+ KB
None
                id
count  4800.000000
mean   2399.500000
std    1385.784976
min       0.000000
25%    1199.750000
50%    2399.500000
75%    3599.250000
max    4799.000000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        1200 non-null   int64 
 1   sentence  1200 non-null   object
dtypes: int64(1), object(1)
memory usage: 18.9+ KB
None
                id
count  1200.000000
mean    599.500000
std     346.554469
min       0.000000
25%     299.750000
50%     599.500000
75%     899.250000


In [29]:
print(training_data.isnull().sum())
print(unlabelled_data.isnull().sum())

id            0
sentence      0
difficulty    0
dtype: int64
id          0
sentence    0
dtype: int64


In [30]:
import re

def clean_text(text):
    text = re.sub(r'\W', ' ', text)  # Remove all non-word characters
    text = re.sub(r'\s+', ' ', text) # Replace multiple spaces with a single space
    return text

training_data['sentence'] = training_data['sentence'].apply(clean_text)
unlabelled_data['sentence'] = unlabelled_data['sentence'].apply(clean_text)


In [ ]:
import re

def clean_text(text):
    # Supprimer les caractères spéciaux et les chiffres
    text = re.sub(r'[^a-zA-ZéèàêâôûùïüçÉÈÀÊÂÔÛÙÏÜÇ\s]', '', text)
    # Convertir en minuscules
    text = text.lower()
    return text

training_data['sentence'] = training_data['sentence'].apply(clean_text)


In [ ]:
!pip install spacy
!python -m spacy download fr_core_news_sm

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

# Load French stopwords
french_stopwords = stopwords.words('french')

vectorizer = TfidfVectorizer(stop_words=french_stopwords, ngram_range=(1, 2))
X_train = vectorizer.fit_transform(training_data['sentence'])



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
from sklearn.model_selection import train_test_split


y = training_data['difficulty']


X_train, X_val, y_train, y_val = train_test_split(X_train, y, test_size=0.2, random_state=42)


In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Initialize the models
logreg = LogisticRegression()
decision_tree = DecisionTreeClassifier()
random_forest = RandomForestClassifier()
knn = KNeighborsClassifier()

# Create a list of models for easy iteration
models = [logreg, decision_tree, random_forest, knn]

# Train each model and evaluate on the validation set
for model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred, average='weighted')
    recall = recall_score(y_val, y_pred, average='weighted')
    f1 = f1_score(y_val, y_pred, average='weighted')
    print(f'Model: {model.__class__.__name__}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-Score: {f1}')


Model: LogisticRegression, Accuracy: 0.425, Precision: 0.4224601544389945, Recall: 0.425, F1-Score: 0.4208469325314973
Model: DecisionTreeClassifier, Accuracy: 0.359375, Precision: 0.35464071241032025, Recall: 0.359375, F1-Score: 0.34987298074421813
Model: RandomForestClassifier, Accuracy: 0.35104166666666664, Precision: 0.41120638740309007, Recall: 0.35104166666666664, F1-Score: 0.33137196055173823
Model: KNeighborsClassifier, Accuracy: 0.17916666666666667, Precision: 0.34723035389018997, Recall: 0.17916666666666667, F1-Score: 0.07616593582821002


In [34]:
from sklearn.model_selection import GridSearchCV

# Example for tuning a RandomForestClassifier
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

CV_rfc = GridSearchCV(estimator=random_forest, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)
print(CV_rfc.best_params_)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomFor

{'criterion': 'gini', 'max_depth': 8, 'max_features': 'sqrt', 'n_estimators': 150}


In [35]:
# Vectorize the unlabelled data sentences
X_unlabelled = vectorizer.transform(unlabelled_data['sentence'])

# Predict using the final model
final_predictions = CV_rfc.predict(X_unlabelled)

# Add predictions to unlabelled data (if needed)
unlabelled_data['predicted_difficulty'] = final_predictions


In [45]:
!pip install streamlit
!streamlit hello


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00



  Welcome to Streamlit. Check out our demo in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.83.234.226:8501

  Ready to create your own Python apps super quickly?
  Head over to https://docs.streamlit.io

  May you create awesome apps!


  Stopping...
  Stopping...


In [54]:
import streamlit as st
import pandas as pd
st.title('Uber pickups in NYC')
st.write("""
# My first app
Hello *world!*
""")
df = pd.read_csv('unlabelled_test_data.csv')
df=df.head(4)
st.line_chart(df)


DeltaGenerator()